* <b> Note: </b> Include this file to the main pipeline to ensure all base path variables are available
* To allow multiple users to run this in parallel, the username is used as the base for defining dir paths, database name etc

In [2]:
#top level dir that holds the demo
dbutils.widgets.text("username", "anindita")
username = dbutils.widgets.get("username")

dbutils.widgets.text("s3Bucket", "delta-streaming-demo")
dbutils.widgets.text("sqsName", "delta-sqs-stream")

s3Bucket = dbutils.widgets.get("s3Bucket")
sqsName = dbutils.widgets.get("sqsName")

In [3]:
#databricks --profile AWS_FIELD_ENG secrets put --scope anindita_scope --key sqs_arn --string-value "https://sqs.us-west-2.amazonaws.com/<ACCOUNT_ID>/delta-sqs-stream"
sqsUrl = dbutils.secrets.get(scope = "anindita_scope", key = "sqs_arn")

In [4]:
dbutils.fs.ls('/demo-datasets/sensor')

Out[3]: [FileInfo(path='dbfs:/demo-datasets/sensor/RUL_FD001.txt', name='RUL_FD001.txt', size=429),
 FileInfo(path='dbfs:/demo-datasets/sensor/test_FD001.txt', name='test_FD001.txt', size=2228855),
 FileInfo(path='dbfs:/demo-datasets/sensor/train_FD001.txt', name='train_FD001.txt', size=3515356)]

In [5]:
#dir that holds a sample dataset
file_source = "/demo-datasets/sensor"
print("Original dataset location=" + file_source)

#database name in which relevant table artifacts will be created
databaseName=username
print("databaseName=" + databaseName)

#Additional join tables

#user specific working dir 
basePath="/"+ username+ "/sensor"
print("basePath=" + basePath)

sleep_time = 1

landing_dir = basePath + "/landing"
refined_dir = basePath + "/refined"
aggregated_dir = basePath + "/aggregated"
print("landing_dir=" + landing_dir + " refined_dir=" + refined_dir + " aggregated_dir="+ aggregated_dir)

bronzeCheckpointPath = "{}/bronze.checkpoint".format(basePath)
bronzeOutPath = "{}/bronze".format(basePath)
print("bronzeCheckpointPath="+ bronzeCheckpointPath + " bronzeOutPath="+ bronzeOutPath)

silverCheckpointPath = "{}/silver.checkpoint".format(basePath)
silverOutPath = "{}/silver".format(basePath)
print("silverCheckpointPath="+ silverCheckpointPath + " silverOutPath="+ silverOutPath)

goldCheckpointPath = "{}/gold.checkpoint".format(basePath)
goldOutPath = "{}/gold".format(basePath)
print("goldCheckpointPath="+ goldCheckpointPath + " goldOutPath="+ goldOutPath)

mount_name = username + "-delta-stream"

In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType

sensor_schema = StructType([
    StructField("unit_num", IntegerType()),
    StructField("cycle_time", IntegerType()),
    StructField("ops_1", DoubleType()),
    StructField("ops_2", DoubleType()),
    StructField("ops_3", DoubleType()),
    StructField("s_1", DoubleType()),
    StructField("s_2", DoubleType()),
    StructField("s_3", DoubleType()),
    StructField("s_4", DoubleType()),
    StructField("s_5", DoubleType()),
    StructField("s_6", DoubleType()),
    StructField("s_7", DoubleType()),
    StructField("s_8", DoubleType()),
    StructField("s_9", DoubleType()),
    StructField("s_10", DoubleType()),
    StructField("s_11", DoubleType()),
    StructField("s_12", DoubleType()),
    StructField("s_13", DoubleType()),
    StructField("s_14", DoubleType()),
    StructField("s_15", DoubleType()),
    StructField("s_16", DoubleType()),
    StructField("s_17", DoubleType()),
    StructField("s_18", DoubleType()),
    StructField("s_19", DoubleType()),
    StructField("s_20", DoubleType()),
    StructField("s_21", DoubleType())
])

engine_metadata_schema = StructType([
    StructField("rul", IntegerType()),
    StructField("engine_id", IntegerType()),
    StructField("engine_name", StringType()),
    StructField("description", StringType())
])